In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Set de datos
Antes de armar la red neuronal vamos a importar los datos que se van a requerir usando pandas. Vamos a hacer un preprocesamiento para que el formato sea compatible con la red neuronal que armaremos

In [ ]:
datos = pd.read_csv("Churn_Modelling.csv")
datos.head()

Imprimiento los primeros 5 elementos podemos ver que nuestra variable destino es la ultima columna **Exited** donde 1 es que se dio de baja y 0 que se quedó.

Es importante que en este paso vayamos pensando que tendremos que hacer con algunas columnas, ya sea eliminarlas, normalizarlas, llevar alguna transformación etc. Por ejemplo:
* RowNumber, CustomerID, Surname no nos dicen nada de utilidad. Predecir si alguien va a darse de baja por su ID de cliente, apellido o numero de renglon en la base de datos no nos va a dar buenos resultados. 
* Las columnas que son categoricas las vamos a transformar con **one hot encoding** (Genero y ubicacion geográfica)

In [ ]:
# Separamos la ultima columna para que sea variable destino
datos_y = datos[datos.columns[-1]]
datos_y.head()


In [ ]:
# Eliminamos las columnas que no funcionarán
datos_x = datos.drop(columns=["RowNumber", "CustomerId", "Surname"])
datos_x.head()

In [ ]:
# Convertimos en one hot encoding las columnas de genero y zona geográfica
datos_x = pd.get_dummies(datos_x)
datos_x = datos_x.drop(columns=["Exited"])
datos_x.head()

Ya se cuenta con una variable que tiene todas las entradas al modelo **datos_x** y otra con la salida **datos_y**. Para datos_x se cuenta con una columna por cada categoria de las variables Genero y Zona geofráfica.

# Dividir datos entre entrenamiento y test

In [ ]:
datos_x.shape[0]

In [ ]:
# X_train = datos_x[:int(datos_x.shape[0]*.8)] 
# X_test = datos_x[:int(datos_x.shape[0]*.2)]
# y_train = datos_y[:int(datos_y.shape[0]*0.8)]
# y_test = datos_y[:int(datos_y.shape[0]*0.2)]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datos_x, datos_y, test_size = 0.2, random_state = 42)

In [ ]:
print("X Train: {}, X Test: {}, y_train: {}, y_test: {}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

In [ ]:
entradas = X_train.shape[1]

# Escalado datos
Ahora vamos a escalar los valores para que esten dentro de un rango mas corto.

In [ ]:
escalador = StandardScaler()
X_train = escalador.fit_transform(X_train)
X_test = escalador.fit_transform(X_test)

In [ ]:
X_train[0]

# Tensores
Para poder procesar los datos en la red neuronal necesitamos que todos los datos estén en tensores, asi que haremos las conversiones necesarias

In [ ]:
t_X_train = torch.from_numpy(X_train).float().to("cpu")
t_X_test = torch.from_numpy(X_test).float().to("cpu")
t_y_train = torch.from_numpy(y_train.values).float().to("cpu")
t_y_test = torch.from_numpy(y_test.values).float().to("cpu")
t_y_train = t_y_train[:,None]
t_y_test = t_y_test[:, None]


In [ ]:
test = TensorDataset(t_X_test, t_y_test)
print(test[0])

In [ ]:
t_y_train

# Estructura de la red neuronal
Ahora vamos a armar una estructura básica de una red neuronal la cual va a recibir los datos de **X** para eventualmente poder predecir **y**

Para hacer esto tenemos que crear una Clase la cual hereda de nn.Module de torch.

In [ ]:
class Network(nn.Module):
    
    def __init__(self, entradas):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(entradas, 15)
        self.linear2 = nn.Linear(15, 8)
        self.linear3 = nn.Linear(8,160)
        self.linear4 = nn.Linear(160, 200)
        self.linear5 = nn.Linear(200, 1)
        # self.linear3 = nn.Linear(8, 1)
    
    def forward(self, xb):
        prediction = torch.sigmoid(input=self.linear1(xb))
        prediction = torch.sigmoid(input=self.linear2(prediction))
        prediction = torch.sigmoid(input=self.linear3(prediction))
        prediction = torch.sigmoid(input=self.linear4(prediction))
        prediction = torch.sigmoid(input=self.linear5(prediction))
        # prediction = torch.sigmoid(input=self.linear3(prediction))
        return prediction

In [ ]:
t_y_test[0]

In [ ]:
#float(len(t_y_test))
#y_pred # Tensor con valores de 0 a 1
#y_pred_class  # Tensor de 0 y 1

t_sum = t_y_test.sum()  #461
t_len = len(t_y_test) #2000
calculo = 416/2000 #0.2305
eq_es = t_y_test.sum()/float(len(t_y_test))    #0.2080
#print("t_y_sum: {}, t_y_len: {}, calculo: {}, eq: {}".format(t_sum, t_len, calculo, eq_es))
#print(y_pred_class.eq(eq_es))
correct = (y_pred_class == t_y_test).sum()
print(correct)
t_y_test
indice = 568
print(y_pred_class[indice], t_y_test[indice])
print("La prediccion es: {}     El valor real: {}".format(model(t_X_test[indice]), t_y_test[indice]))

In [ ]:
%%time

lr = 0.01
model = Network(entradas=entradas)
print(f"Model Architecture:\n{model}")
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

nb_epochs = 1200
print_offset = 100

df_tracker = pd.DataFrame()
print("\nTraining the model...")
for epoch in range(1, nb_epochs+1):
    y_pred = model(t_X_train)
    #print(y_pred.shape)
    loss = criterion(input=y_pred, target=t_y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % print_offset == 0:
        print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")
    
    # Print test-accuracy after certain number of epochs
    with torch.no_grad():
        y_pred = model(t_X_test)
        y_pred_class = y_pred.round()
        #print(y_pred_class.shape)
        # accuracy = y_pred_class.eq(t_y_test).sum() / float(len(t_y_test))
        correct = (y_pred_class == t_y_test).sum()
        accuracy = 100 * correct / float(len(t_y_test))
        if epoch % print_offset == 0:
            print("Pred_Clas: {}, Y_test".format(t_y_test.sum(), len(t_y_test)))
            print(f"Accuracy (on test-set): {round(accuracy.item(), 4)}")
    
    df_temp = pd.DataFrame(data={
        'Epoch': epoch,
        'Loss': round(loss.item(), 4),
        'Accuracy': round(accuracy.item(), 4)
    }, index=[0])
    df_tracker = pd.concat(objs=[df_tracker, df_temp], ignore_index=True, sort=False)

print(f"\nFinal Accuracy (on test-set): {round(accuracy.item(), 4)}")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(df_tracker['Epoch'], df_tracker['Loss'], color='purple', linewidth=2, label='Loss')
plt.title("Loss over time", fontsize=25)
plt.xlabel("Epoch", fontsize=16)
plt.ylabel("Loss", fontsize=16)
plt.grid()
plt.legend(loc='best', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_tracker['Epoch'], df_tracker['Accuracy'], color='blue', linewidth=2, label='Accuracy')
plt.title("Accuracy over time", fontsize=25)
plt.xlabel("Epoch", fontsize=16)
plt.ylabel("Accuracy", fontsize=16)
plt.grid()
plt.legend(loc='best', fontsize=16)
plt.show()